In [19]:
import weaviate

client = weaviate.connect_to_local()

print(client.is_ready())

client.close()

True


In [ ]:
from weaviate.classes.config import Configure

client = weaviate.connect_to_local()

questions = client.collections.create(
    name="Question",
    vector_config=Configure.Vectors.text2vec_ollama(
        api_endpoint="http://host.docker.internal:11434",
        model="nomic-embed-text"
    )
)

client.close()

In [21]:
import requests
import json

client = weaviate.connect_to_local()

resp = requests.get(
    "https://raw.githubusercontent.com/weaviate-tutorials/quickstart/main/data/jeopardy_tiny.json"
)
data = json.loads(resp.text)

questions = client.collections.use("Question")

with questions.batch.fixed_size(batch_size=200) as batch:
    for d in data:
        batch.add_object(
            {
                "answer": d["Answer"],
                "question": d["Question"],
                "category": d["Category"],
            }
        )
        if batch.number_errors > 10:
            print("Batch import stopped due to excessive errors.")
            break

failed_objects = questions.batch.failed_objects
if failed_objects:
    print(f"Number of failed imports: {len(failed_objects)}")
    print(f"First failed object: {failed_objects[0]}")

client.close()  # Free up resources

In [22]:
client = weaviate.connect_to_local()

questions = client.collections.use("Question")

response = questions.query.near_text(
    query="biology",
    limit=2
)

for obj in response.objects:
    print(json.dumps(obj.properties, indent=2))

client.close()     


{
  "question": "In 1953 Watson & Crick built a model of the molecular structure of this, the gene-carrying substance",
  "category": "SCIENCE",
  "answer": "DNA"
}
{
  "question": "This organ removes excess glucose from the blood & stores it as glycogen",
  "category": "SCIENCE",
  "answer": "Liver"
}


In [23]:
from weaviate.classes.generate import GenerativeConfig

client = weaviate.connect_to_local()

questions = client.collections.use("Question")

response = questions.generate.near_text(
    query="biology",
    limit=2,
    grouped_task="Write a tweet with emojis about these facts.",
    generative_provider=GenerativeConfig.ollama(
        api_endpoint="http://host.docker.internal:11434",
        model="llama3.2"
    )
)

In [24]:
print(response.generative.text)

client.close()

"Did you know? 🤯 DNA is the molecule that holds our genes, built by Watson & Crick in 1953! 💡 And did you know about the liver's role? 🧬 It removes excess glucose from the blood & stores it as glycogen! #ScienceFacts #Genetics #LiverFunctions"
